# 15-12-30

David Rowe gave the following explanation for his centroid finding in PS3: (**My comments in bold**)

"
I have a routine SimplePhotometry(X, Y, ApRadius, Image, Stats) that calculates the centroid, flux, background, and a number of other statistics from Image, given a location on the image (X,Y) and the radius of the aperture (ApRadius). The calculated statistics are contained in the structure called Stats. For peak tracking I call this routine 4 times using the previously calculated values of the centroid as the new location. In other words, the routine is as follows:

Given Xm, Ym as the mouse location

SimplePhotometry(Xm,Ym,ApRadius,Image,Stats)
for i = 0 to 3
	SimplePhotometry(Stats.Xcen,Stats.Ycen,ApRadius,Image,Stats)
next 

(Xcen,Ycen) is the calculated centroid.

** Centroid calculated for mouse location, then centroid is calculated several more times with centroid as center of calculation, within circular aperture**

This causes the circular aperture to be better and better centered on the peak in the image. Why so many iterations? For empirical reasons. I found that five total calls was needed in marginal cases.

I've included the VB.Net code for SimplePhotometry() if you're interested.

I hope this answers your question. If not, let me know.

My best,
Dave


    Public Shared Sub SimplePhotometry(ByVal X As Double, ByVal Y As Double, ByVal ApRad As Double, ByVal ImData As ImageLib.ImageType, ByRef Stats As StarStatsType)

        ' calculate the statistics for star located at pixel location X,Y in image ImData
        ' results are returned in Stats
        ' ApRad is the radius of the object aperture, in pixels
        ' the background is not considered

        Dim Ip0, Ip1, Jp0, Jp1 As Integer
        Dim Val As Double
        Dim S, Sx, Sy, Sx2, Sy2, Sxy, Arg, MaxVal As Double
        Dim Xcen, Ycen, R, R2 As Double
        Dim X2, Y2, Xy, R2x, R2y, Sr2X, Sr2Y As Double
        Dim Theta, u2, V2, AR, Sigma As Double
        Dim Px, Py As Integer
        Dim N As Integer
        Dim B, Den As Double

** Center of centroid region defined by mouse location ** 

        Xcen = X
        Ycen = Y

** Size of input image saved **

        Px = ImData.N1
        Py = ImData.N2

        'aperture stats

** Calculating pixel indices surrounding the aperture to be looped through **

        Ip0 = Math.Max(0, Math.Round(Xcen - (ApRad + 0.5)))
        Ip1 = Math.Min(Px - 1, Math.Round(Xcen + (ApRad + 0.5)))
        Jp0 = Math.Max(0, Math.Round(Ycen - (ApRad + 0.5)))
        Jp1 = Math.Min(Py - 1, Math.Round(Ycen + (ApRad + 0.5)))
        
        ' first find centroid
        S = 0 : Sx = 0 : Sy = 0
        
** Loop through square of incices containing the aperture **
        
        For i = Ip0 To Ip1
            For j = Jp0 To Jp1
            
** Check that present pixel is within radius of aperture **            
         
                R = Math.Sqrt((Xcen - i) ^ 2 + (Ycen - j) ^ 2)
                If R <= ApRad Then
                    Val = ImData.Data(i, j)
                    
** Accumulating values for centroid calculation **
                    
                    S = S + Val
                    Sx = Sx + Val * i
                    Sy = Sy + Val * j
                End If
            Next j
        Next i

** Calculating centroid values **

        If S > 0 Then
            Xcen = Sx / S
            Ycen = Sy / S
        Else
            Xcen = 0
            Ycen = 0
        End If

** Assign calculated centroid **

        Stats.Xcen = Xcen
        Stats.Ycen = Ycen
        Stats.Total = S
        ' find moments about this centroid
        N = 0 : S = 0 : Sx2 = 0 : Sy2 = 0 : Sxy = 0 : X2 = 0 : Y2 = 0 : Xy = 0 : Sr2X = 0 : Sr2Y = 0 : MaxVal = 0
        For i = Ip0 To Ip1
            For j = Jp0 To Jp1
                R = Math.Sqrt((Xcen - i) ^ 2 + (Ycen - j) ^ 2)
                If R <= ApRad Then
                    Val = ImData.Data(i, j)
                    MaxVal = Math.Max(MaxVal, Val)
                    S = S + Val
                    
** Calculating Central Moments **

*mu_10*

                    Sx2 = Sx2 + Val * (i - Xcen) ^ 2
*mu_01*
                    
                    Sy2 = Sy2 + Val * (j - Ycen) ^ 2
*Mu_11*                    
                    
                    Sxy = Sxy + Val * (i - Xcen) * (j - Ycen)
** Not sure what the rest of this is for **                    
                    
                    Sr2X = Sr2X + Val * (i - Xcen) * ((i - Xcen) ^ 2 + (j - Ycen) ^ 2)
                    Sr2Y = Sr2Y + Val * (j - Ycen) * ((i - Xcen) ^ 2 + (j - Ycen) ^ 2)
                    N = N + 1
                End If
            Next j
        Next i

        If S > 0 Then
            X2 = Sx2 / S
            Y2 = Sy2 / S
            Xy = Sxy / S
            R2x = Sr2X / S
            R2y = Sr2Y / S
            If (X2 + Y2) > 0 Then
                R2x = R2x / (X2 + Y2) ^ 1.5
                R2y = R2y / (X2 + Y2) ^ 1.5
            Else
                R2x = 0
                R2y = 0
            End If
        End If

        Arg = X2 + Y2
        If Arg > 0 Then Stats.RMS = Math.Sqrt((X2 + Y2) / 2.0) Else Stats.RMS = 0

        Theta = Math.Atan2(2 * Xy, X2 - Y2) / 2.0
        u2 = X2 * Math.Cos(Theta) ^ 2 + Y2 * Math.Sin(Theta) ^ 2 + 2 * Xy * Math.Sin(Theta) * Math.Cos(Theta)
        V2 = Y2 * Math.Cos(Theta) ^ 2 + X2 * Math.Sin(Theta) ^ 2 - 2 * Xy * Math.Sin(Theta) * Math.Cos(Theta)
        If u2 > 0 And V2 > 0 Then AR = Math.Sqrt(u2 / V2) Else AR = 0

        Stats.AR = AR
        Stats.Theta = Theta
        Stats.U2 = u2
        Stats.V2 = V2
        Stats.X2 = X2
        Stats.Y2 = Y2
        Stats.Xy = Xy
        Stats.R2x = R2x
        Stats.R2y = R2y
        Stats.Max = MaxVal

        ' -------------------------------------------------------------------------
        ' fit radial profile to a Gaussian

        Ip0 = Math.Max(0, Math.Round(Xcen - (ApRad + 0.5)))
        Ip1 = Math.Min(Px - 1, Math.Round(Xcen + (ApRad + 0.5)))
        Jp0 = Math.Max(0, Math.Round(Ycen - (ApRad + 0.5)))
        Jp1 = Math.Min(Py - 1, Math.Round(Ycen + (ApRad + 0.5)))

        S = 0 : Sx = 0 : Sy = 0 : N = 0 : Sxy = 0 : Sx2 = 0
        For i = Ip0 To Ip1
            For j = Jp0 To Jp1
                R2 = (Xcen - i) ^ 2 + (Ycen - j) ^ 2
                R = Math.Sqrt(R2)
                If R <= ApRad Then
                    Val = ImData.Data(i, j)
                    If Val > 0.1 * MaxVal Then
                        Val = Math.Log(Val)
                        N += 1
                        Sy = Sy + Val
                        Sx = Sx + R2
                        Sx2 = Sx2 + R2 * R2
                        Sxy = Sxy + Val * R2
                    End If
                End If
            Next j
        Next i
        B = 0
        Den = Sx * Sx - N * Sx2
        If Den <> 0 Then B = (Sx * Sy - N * Sxy) / Den
        Sigma = 0
        If B < 0 Then Sigma = Math.Sqrt(-1.0 / (2.0 * B))
        Stats.FWHMGauss = 2.35 * Sigma

        '----------------------------------------------------------------------------

        '----------------------------------------------------------------------------
        ' fit to a Moffat Distribution, f(r) = (alfa + beta*r^2)^-MoffExp

        Dim alfa, beta, Rmoff As Double
        Dim MoffExp, MoffExpInv As Double

        MoffExp = 2.5
        MoffExpInv = 1.0 / MoffExp

        S = 0 : Sx = 0 : Sy = 0 : N = 0 : Sxy = 0 : Sx2 = 0
        For i = Ip0 To Ip1
            For j = Jp0 To Jp1
                R2 = (Xcen - i) ^ 2 + (Ycen - j) ^ 2
                R = Math.Sqrt(R2)
                If R <= ApRad Then
                    Val = ImData.Data(i, j)
                    If Val > 0.1 * MaxVal Then
                        Val = (MaxVal / Val) ^ MoffExpInv
                        N += 1
                        Sy = Sy + Val
                        Sx = Sx + R2
                        Sx2 = Sx2 + R2 * R2
                        Sxy = Sxy + Val * R2
                    End If
                End If
            Next j
        Next i
        beta = 0
        Den = Sx * Sx - N * Sx2
        If Den <> 0 Then beta = (Sx * Sy - N * Sxy) / Den
        If N > 0 Then alfa = (Sy - beta * Sx) / N
        If alfa <> 0 Then
            beta = beta / alfa
            alfa = 1.0
        End If
        If beta > 0 Then Rmoff = (1.0 / beta) ^ 0.5
        Stats.FWHMmoff = Rmoff * 2.0 * (2.0 ^ MoffExpInv - 1.0) ^ 0.5
        '---------------------------------------------------------------------------

    End Sub
    
"

I see that Dave's procedure recursively calculates the centroid of the aperture, using the last centroid location as the center of the next search's mask. 

I can imitate this behavior with a combination of my work so far. This would be an automatic solution
1. Find contours of side lobes
2. Calculate centroid of side lobe (masked by contour)
3. Calculate circle mask centered on centroid that fits insie contour
4. Calculate centroid with circle mask
5. Use new centroid location as center to calculate circular mask again
6. Repeat steps 4-5 3x

For a manual solution, I can imitate the PS3 behavior
1. User selects the aperture diameter and location (with mouse or arrow keys)
2. Calculate centroid with circle mask
3. Use new centroid location as center of new circular mask (same diameter as before)
4. Repeat steps 2-3 3x

In [ ]:
# Re-doing my auto centroid finding using the automatic solution explained above

